In [ ]:
import requests
import re
import json
from tqdm import tqdm

# Metacritic API url
url = "https://mcgqlapi.com/api"
 
# Using readlines()
movies_file = open('movies.txt', 'r')
movies_lines = movies_file.readlines()

movies = {}

# Obtain information using metacritic API
# https://mcgqlapi.com/docs/movieinfo.doc.html
for movie_line in tqdm(movies_lines, desc='Querying MetacriticAPI...', total=len(movies_lines)): 
    title, year = re.findall('(.*) \((.*)\)', movie_line)[0]
    query = (
      'query {',
      'movie(input: {',
      f'  title: "{title}",',
      f'  year: "{year}"',
      '}) {',
      '  title',
      '  criticScore',
      '  year',
      '  director',
      '  releaseDate',
      '  genres',
      '  cast',
      '  rating',
      '  runtime',
      '  summary',
      '  numOfCriticReviews',
      ' }',
      '}'
    )
    query = '\n'.join(query)
    response = requests.post(url=url, json={"query": query})
    if response.status_code == 200:
        movies[title] = response.json()

with open('movies.json', 'w') as output_file:
     output_file.write(json.dumps(movies))

Querying MetacriticAPI...:  82%|███████████▌  | 246/299 [01:35<00:18,  2.93it/s]

In [3]:
import json
import pandas as pd
import csv


# Abra o arquivo JSON e carregue o conteúdo em uma variável
with open('movies.json', 'r') as file:
    data = json.load(file)

with open('movies.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['title', 'criticScore', 'year', 'director', 'releaseDate', 'genres', 'cast', 'rating', 'runtime', 'summary', 'numOfCriticReviews'])
    
    for movie_title, movie_data in data.items():
        movie = movie_data['data']
        if((movie.get('movie'))):
            writer.writerow([
                movie['movie']['title'],
                movie['movie']['criticScore'],
                movie['movie']['year'],
                movie['movie']['director'],
                movie['movie']['releaseDate'],
                movie['movie']['genres'],
                movie['movie']['cast'],
                movie['movie']['rating'],
                movie['movie']['runtime'],
                movie['movie']['summary'],
                movie['movie']['numOfCriticReviews']
            ])

In [4]:
pd.read_csv('movies.csv')

,title,criticScore,year,director,releaseDate,genres,cast,rating,runtime,summary,numOfCriticReviews
0,The Godfather,10,1972,['Francis Ford Coppola'],"March 24, 1972",['Crime'],"['Al Pacino', 'Marlon Brando']",R,175 min,Francis Ford Coppola's epic features Marlon Br...,16
1,The Shawshank Redemption,81,1994,['Frank Darabont'],"September 1, 1994",['Crime'],"['Morgan Freeman', 'Tim Robbins']",R,142 min,"Wrongly convicted, Andy Dufresne (Tim Robbins)...",21
2,The Dark Knight,84,2008,['Christopher Nolan'],"July 18, 2008",['Crime'],"['Aaron Eckhart', 'Christian Bale', 'Heath Led...",PG-13,152 min,Batman raises the stakes in his war on crime. ...,39
3,The Lord of the Rings: The Fellowship of the Ring,92,2001,['Peter Jackson'],"December 19, 2001",['Fantasy'],"['Andy Serkis', 'Elijah Wood']",PG-13,178 min,NaN,34
4,Pulp Fiction,94,1994,['Quentin Tarantino'],"October 14, 1994",['Crime'],"['Bruce Willis', 'John Travolta', 'Samuel L. J...",R,154 min,NaN,24
5,Star Wars: Episode IV - A New Hope,90,1977,['George Lucas'],"May 25, 1977",['Fantasy'],"['Carrie Fisher', 'Harrison Ford', 'Mark Hamill']",PG,121 min,"A long time ago in a galaxy far, far away... P...",24
6,Forrest Gump,82,1994,['Robert Zemeckis'],"July 6, 1994",['Romance'],"['Gary Sinise', 'Robin Wright', 'Sally Field',...",PG-13,142 min,NaN,20
7,Inception,74,2010,['Christopher Nolan'],"July 16, 2010",['Thriller'],"['Elliot Page', 'Joseph Gordon-Levitt', 'Ken W...",PG-13,148 min,"Dom Cobb is a skilled thief, the absolute best...",42
8,The Lord of the Rings: The Return of the King,94,2003,['Peter Jackson'],"December 17, 2003",['Fantasy'],"['Andy Serkis', 'Orlando Bloom']",PG-13,201 min,Sauron's forces have laid siege to Minas Tirit...,41
